<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(4)_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ノートブック (4) RAG システムの構築とデータ検索の実装

## 1. RAG (Retrieval-Augmented Generation) の技術的背景
大規模言語モデル (LLM) は、事前学習に含まれない「非公開データ（社内規定など）」や「最新情報」に関する質問には正確に答えることができません。無理に答えさせると、事実に基づかない回答（ハルシネーション）を生成するリスクがあります。

**RAG (検索拡張生成)** は、この課題を解決するために、以下のプロセスを統合したアーキテクチャです。

1.  **Retrieval (検索)**: ユーザーの質問 (Query) をベクトル化し、データベースから関連するドキュメント (Context) を意味的類似度に基づいて抽出します。
2.  **Augmentation (拡張)**: 抽出したドキュメントを「正解の根拠」としてプロンプトに埋め込みます。
3.  **Generation (生成)**: LLM は、与えられた根拠のみに基づいて回答を生成します。

本ノートブックでは、架空の商社 **「株式会社静浜商事」** の社内規定データベースを構築し、LangGraph を用いて検索と回答生成のフローを実装します。

In [ ]:
# 必要なライブラリのインストール
# langchain-chroma: ベクトルデータベース ChromaDB のラッパー
# sentence-transformers: テキストをベクトルに変換する埋め込みモデル
!pip install -qU langgraph langchain-openai langchain-community langchain-chroma sentence-transformers termcolor

## 2. 環境設定と推論モデルの定義

RAG の「生成 (Generate)」パートを担当する LLM を定義します。
ここでは、OpenRouter 経由で **`nvidia/nemotron-3-nano-30b-a3b`** を使用します。このモデルはコンテキスト（与えられた文章）の理解力が高く、指示に従って忠実に回答する能力に優れています。

また、API キーの読み込み処理にはエラーハンドリングを実装し、設定漏れがある場合に警告を表示するようにしています。

In [ ]:
# 2. 環境設定とモデルの準備
import os
import sys
from google.colab import userdata
from termcolor import colored
from langchain_openai import ChatOpenAI

# APIキーの読み込み
try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    print(colored("OPENROUTER_API_KEY loaded.", "green"))
except Exception:
    print(colored("Error: OPENROUTER_API_KEY not found in Secrets.", "red"))

# モデル定義 (安定性の高い Nemotron を使用)
MODEL_NAME = "nvidia/nemotron-3-nano-30b-a3b"

llm = ChatOpenAI(
    model=MODEL_NAME,
    openai_api_base="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    streaming=True,
    model_kwargs={
        "extra_body": {
            "provider": {
                "order": ["deepinfra"],
                "allow_fallbacks": False
            }
        }
    }
)

## 3. 知識ベースのデータ構造とチャンク分割

### 📝 テキストデータの定義と前処理
RAG において、長いドキュメントをそのまま扱うことは、LLM の入力トークン制限や検索精度の観点から推奨されません。そのため、前処理としてテキストを適切なサイズ（チャンク）に分割します。

ここでは、仮想企業「株式会社静浜商事」の社内規定データを定義し、それがどのように分割されるかをシミュレーションします。

In [ ]:
import textwrap
from termcolor import colored

# 検索対象となるテキストデータの定義
rag_source_text = """
【株式会社静浜商事 (Shizuhama Trading Co., Ltd.) 社内規定・概要】

[1. 企業概要]
本社：静岡県静岡市葵区御幸町 1-1
代表：山田 健太郎
事業内容：
- 静岡県産茶葉の輸出、および緑茶成分を使用した化粧品開発
- EV（電気自動車）向け軽量部品の調達・卸売
- 地域密着型再生可能エネルギー（小水力発電）のコンサルティング

[2. 勤務体系・リモートワーク規定]
- 基本就業時間：9:00 - 18:00
- フレックス制度：「富士山フレックス」を導入。コアタイムは 11:00 - 14:00 とする。
- リモートワーク：全部署で推奨。週5日のフルリモートも可とするが、月1回の「全体戦略会議」のみ出社必須とする。
- ワーケーション：伊豆半島および浜名湖エリアの提携施設での勤務を認める。

[3. 出張・経費規定]
- 新幹線：片道100km以上で指定席利用可。役員以外はグリーン車利用不可。
- 宿泊費：国内一律 10,000円/泊、海外（アジア）15,000円/泊、海外（欧米）20,000円/泊を上限とする。
- 自家用車通勤：静岡本社は不可。浜松支店、沼津営業所は許可制とする。

[4. 独自の福利厚生・休暇]
- 茶摘み休暇：毎年4月下旬〜5月、実家または契約農園での茶摘みを手伝う社員に最大3日間の特別有給を付与する。
- プラモデル製作休暇：ものづくり文化の理解促進のため、年1回、模型製作イベント参加日を公休扱いとする。
- サウナ手当：健康増進のため、月額3,000円までサウナ利用料を補助する（「しきじ」等の利用を推奨）。
"""

print(colored(f"原稿データの文字数: {len(rag_source_text)} 文字", "cyan"))

# チャンク分割のシミュレーション（可視化）
# 実際には LangChain の Splitter を使いますが、ここでは構造理解のために簡易表示します
sections = rag_source_text.strip().split('\n\n')
print(colored("\n--- チャンク分割イメージ (論理的なまとまりでの分割例) ---", "yellow"))
for i, section in enumerate(sections):
    preview = textwrap.shorten(section.replace('\n', ' '), width=60, placeholder="...")
    print(f"Chunk {i+1}: {preview}")

## 4. ベクトル化と検索空間の構築

### 📊 ベクトルデータベース (Vector Store) の作成
コンピュータはテキストの意味を直接理解できません。そこで、テキストを数百〜数千次元の数値の列（ベクトル）に変換します。これを **Embedding (埋め込み)** と呼びます。

RAG における「検索」は、以下の数学的な操作に基づきます。

1.  ドキュメント $D$ をベクトル $\vec{d}$ に変換し、データベースに保存する。
2.  ユーザーの質問 $Q$ を同じモデルでベクトル $\vec{q}$ に変換する。
3.  ベクトル空間上で、$\vec{q}$ と最も距離が近い（角度が似ている）$\vec{d}$ を抽出する。

類似度の計算には、一般的に **コサイン類似度 (Cosine Similarity)** が用いられます。

$$\text{similarity} = \cos(\theta) = \frac{\vec{q} \cdot \vec{d}}{\|\vec{q}\| \|\vec{d}\|}$$

以下では、実際にデータベースを構築します。

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# (1) テキストの分割 (Chunking)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,    # 1つのデータの文字数
    chunk_overlap=50   # 文脈を維持するための重複幅
)
docs = splitter.split_documents([Document(page_content=rag_source_text)])

print(colored(f"分割されたドキュメント数: {len(docs)} 件", "green"))

# (2) ベクトル化モデルの準備
print("Embedding モデルをロード中...")
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

# (3) DBへの保存 (重複防止処理付き)
# 一度DBインスタンスを作成し、もし古いデータがあれば削除してから再登録します
vector_store = Chroma(
    collection_name="shizuhama_regulations",
    embedding_function=embeddings,
    # 永続化ディレクトリを指定しない場合はメモリ上で動作しますが、
    # セッション内での重複を防ぐために明示的な初期化を行います。
)

# 既存データの全削除（リセット）
try:
    vector_store.delete_collection()
    # 削除後に再定義
    vector_store = Chroma(
        collection_name="shizuhama_regulations",
        embedding_function=embeddings,
    )
except:
    pass # 初回実行時など、削除対象がない場合はスルー

# データの追加
vector_store.add_documents(docs)

print(colored("ベクトルデータベース構築完了（リセット済み）。", "green"))

### 🔍 検索精度の検証 (Retrieval Check)

RAG の回答精度は「検索精度」に依存します。LLM に渡す前に、意図したドキュメントが正しく検索（Retrieve）できているかを確認することは重要です。

以下のセルを実行すると入力フォームが表示されます。質問を入力し、実際にどのドキュメントがヒットし、その **スコア (距離)** がどれくらいかを確認してください。スコアが低い（距離が遠い）場合、LLM は正しい根拠を得られません。

* **検証用クエリ例**:
    * `フレックス制度について`
    * `新幹線の利用規定`
    * `東京支社の住所` （存在しない情報の確認）

In [ ]:
# @title 検索実験フォーム
# @markdown 検索したい質問を入力して実行してください。

query = "東京への出張のときは、旅費が支出されますか？" # @param {type:"string"}
import matplotlib.pyplot as plt
import numpy as np

# 類似度検索の実行 (スコア付き)
# k=3 は上位3件を取得するという意味です
results_with_score = vector_store.similarity_search_with_score(query, k=3)

print(colored(f"検索クエリ: {query}", "yellow", attrs=["bold"]))
print("-" * 60)

scores = []
contents = []

for i, (doc, score) in enumerate(results_with_score):
    # Chroma の score は L2 distance (距離) なので、値が小さいほど類似度が高い
    # (一般的なコサイン類似度とは逆の指標であることに注意)
    color = "cyan" if i == 0 else "white"
    print(colored(f"【Rank {i+1}】 Distance Score: {score:.4f}", color, attrs=["bold"]))
    print(f"Content: {doc.page_content.replace(chr(10), ' ')}") # 改行を除去して表示
    print("-" * 60)

    scores.append(score)
    contents.append(f"Rank {i+1}")

# 距離スコアの可視化
if len(scores) > 0:
    plt.figure(figsize=(8, 2))
    plt.barh(contents[::-1], scores[::-1], color='skyblue')
    plt.xlabel('L2 Distance (Lower is Better)')
    # 日本語文字化け回避のためタイトルを英語固定に変更
    plt.title('Similarity Scores')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()

## 5. LangGraph による RAG ワークフローの構築

### ⛓️ ノードとグラフの定義
検索されたテキスト（Context）を LLM に渡し、回答を生成するフローを構築します。
単純な関数呼び出しではなく、LangGraph を用いて「状態 (State)」と「ノード (Node)」によるグラフ構造として定義します。

**データフロー:**
1.  **Start**: ユーザーの質問 `question` が入力される。
2.  **Retrieve Node**:
    * `question` を元に Vector DB を検索。
    * 結果を文字列として結合し、`context` に保存。
    * **ポイント**: ここで検索された中身を可視化します。
3.  **Generate Node**:
    * `question` と `context` をプロンプトに埋め込む。
    * LLM が `answer` を生成。
4.  **End**: 最終的な `answer` を出力。

In [ ]:
from typing import TypedDict
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END

# --- 1. State (状態) の定義 ---
# 各ノード間で受け渡されるデータのスキーマ
class RagState(TypedDict):
    question: str   # ユーザーの入力
    context: str    # 検索された情報（Retrieveノードで更新）
    answer: str     # LLMの回答（Generateノードで更新）

# --- 2. ノード関数の定義 ---

def retrieve_node(state: RagState):
    """検索を実行し、コンテキストを取得するノード"""
    question = state["question"]
    print(colored(f"\n[Process 1] 文書検索を実行中... Query: {question}", "grey"))

    # ベクトルDBをRetrieverインターフェースとして使用
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    found_docs = retriever.invoke(question)

    # 複数の検索結果を一つの文字列に結合
    context_text = "\n\n".join([d.page_content for d in found_docs])

    # 【重要】LLMに渡される生のコンテキストを表示（途中経過の可視化）
    print(colored(f"[Result 1] 検索されたコンテキスト:\n{'-'*20}\n{context_text}\n{'-'*20}", "blue"))

    return {"context": context_text}

def generate_node(state: RagState):
    """コンテキストに基づいて回答を生成するノード"""
    question = state["question"]
    context = state["context"]

    print(colored(f"[Process 2] LLMによる回答生成を実行中...", "grey"))

    # RAG特有のプロンプト（グラウンディング）
    # Context (事実) と Question (質問) を明確に区別して入力します
    prompt = f"""
    あなたは株式会社静浜商事の社内AIアシスタントです。
    以下の【社内規定】のみに基づいて、社員の質問に答えてください。
    規定に記載がない場合は、正直に「規定に記載がありません」と答えてください。
    自身の知識で補完したり、情報を捏造してはいけません。

    【社内規定】
    {context}

    【社員の質問】
    {question}
    """

    # モデル定義は第2章で作成した `llm` インスタンスを使用
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"answer": response.content}

# --- 3. グラフの構築 ---
workflow = StateGraph(RagState)

# ノードの登録
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("generate", generate_node)

# エッジ（接続）の定義
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# コンパイル
app = workflow.compile()
print(colored("RAG ワークフローの構築完了。", "green"))

# グラフ構造の可視化 (Mermaid記法)
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph visualization skipped (requires graphviz).")

## 6. システムの総合動作確認

### ✅ シナリオ別テスト
構築したパイプライン全体を通して質問を行います。
RAG が適切に機能しているか、以下の観点で確認します。

1.  **Fact Retrieval**: 記述されている事実を正しく引き出せるか。
2.  **Synthesis**: 複数のチャンクにまたがる情報を統合できるか。
3.  **Negative Constraint**: 記述されていない情報に対して「分からない」と答えられるか（ハルシネーションの抑制）。

In [ ]:
def ask_bot(question):
    """RAGシステムに質問を投げるヘルパー関数"""
    inputs = {"question": question, "context": "", "answer": ""}

    print("=" * 80)
    print(colored(f"Q. {question}", "yellow", attrs=["bold"]))

    # グラフの実行
    # stream=False で一括実行結果を取得
    result = app.invoke(inputs)

    print(colored("\n[Final Answer] AIの回答:", "cyan", attrs=["bold"]))
    print(result["answer"])
    print("=" * 80 + "\n")

# --- 検証シナリオ ---

# 1. 単純な事実確認
ask_bot("フレックス制度のコアタイムを教えて")

# 2. 複数の情報が必要な質問（休暇制度）
ask_bot("ユニークな休暇制度にはどのようなものがありますか？")

# 3. 規定外の質問（ハルシネーション抑制テスト）
ask_bot("大阪支店の住所はどこですか？")

## 7. 🎓 【演習】PDFデータを用いたRAGシステムの構築

ここまでの学習内容を応用し、実際の PDF ファイル（論文、マニュアル、契約書など）を知識ベースとする検索システムを構築します。
実務においては、テキストデータが綺麗な文字列で用意されていることは稀で、PDF や Word ファイルから情報を抽出するプロセスが必須となります。

### 📦 1. ライブラリの準備とファイルアップロード
PDF の読み込みには `pypdf` ライブラリを使用します。
以下のコードを実行し、手元の PDF ファイルをアップロードしてください。

※ アップロードするファイルがない場合は、Web上の公開PDFなどを適当にダウンロードして使用してください。

In [ ]:
# PDF処理用ライブラリのインストール
!pip install -qU pypdf

import os
from google.colab import files
from termcolor import colored

print(colored("PDFファイルをアップロードしてください...", "cyan"))

# ファイルアップロード機能の呼び出し
uploaded = files.upload()

# アップロードされたファイル名を取得
if uploaded:
    pdf_file_name = list(uploaded.keys())[0]
    print(colored(f"\nファイル '{pdf_file_name}' のアップロードが完了しました。", "green"))
else:
    print(colored("\nファイルがアップロードされませんでした。", "red"))
    pdf_file_name = None

### 🧠 2. 演習：データパイプラインの構築 (穴埋め)

アップロードした PDF を検索可能なベクトルデータベースに変換します。以下のコードの `■■■` の部分を埋めて、パイプラインを完成させてください。

#### 💡 ヒント: 使用する主要クラスとメソッド

1.  **`PyPDFLoader`**: ファイルパスを指定して PDF を読み込みます。`load()` メソッドでドキュメントリストを取得できます。
2.  **`RecursiveCharacterTextSplitter`**: 長い文章を、文脈を考慮しつつ適切な長さに分割するクラスです。
3.  **`Chroma.from_documents`**:
    * **機能**: ドキュメントのリストと埋め込みモデル（Embedding）を受け取り、ベクトルデータベースを構築してインスタンスを返します。
    * **引数**: `documents`（分割されたドキュメントのリスト）, `embedding`（埋め込みモデル）
    * **役割**: テキストのベクトル化とインデックス作成を一括で行う便利なメソッドです。

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# ファイルがアップロードされている場合のみ実行
if pdf_file_name:
    print(f"処理対象: {pdf_file_name}")

    # 1. PDFローダーの初期化とデータのロード
    loader = PyPDFLoader(pdf_file_name)
    raw_docs = loader.load()
    print(f"ロード完了: {len(raw_docs)} ページ")


    # 2. テキスト分割器 (Splitter) の定義 [演習]
    # 長いテキストをチャンクに分割するクラスを定義します。
    # クラス名は Recursive... で始まります。
    text_splitter = ■■■(
        chunk_size=300,    # 1つのチャンクの文字数
        chunk_overlap=50   # 重複させる文字数
    )

    # ドキュメントの分割実行
    pdf_docs = text_splitter.split_documents(raw_docs)
    print(f"チャンク分割完了: {len(pdf_docs)} 件")


    # 3. 埋め込みモデルの準備
    # (前の章と同じ multilingual-e5-large を使用します)
    print("Embedding モデルをロード中...")
    embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


    # 4. ベクトルデータベースの構築 [演習]
    # 分割したドキュメント(pdf_docs)と埋め込みモデル(embeddings)を使ってDBを作成します。
    # Chroma のクラスメソッド from_... を使用します。
    print("ベクトル化を実行中...")

    pdf_vector_store = Chroma.■■■(
        documents=■■■,
        embedding=■■■
    )

    print(colored("PDFデータのデータベース化が完了しました。", "green"))

else:
    print(colored("エラー: PDFファイルが見つかりません。前のセルでアップロードしてください。", "red"))

### 💬 3. 検索テスト

構築したデータベース (`pdf_vector_store`) に対して検索を行い、PDF の内容が正しく取得できるか確認します。
ここでは RAG の全フロー（生成まで）ではなく、最も重要な「検索（Retrieval）」部分を検証します。

In [ ]:
# @title 検索テストフォーム
# @markdown PDFの内容に関する質問を入力してください。

query_for_pdf = "\u3053\u306EPDF\u306E\u8981\u70B9\u3092\u6559\u3048\u3066" # @param {type:"string"}

if 'pdf_vector_store' in globals() and pdf_file_name:
    print(colored(f"Q. {query_for_pdf}", "yellow", attrs=["bold"]))
    print("-" * 60)

    # 類似度検索の実行
    results = pdf_vector_store.similarity_search(query_for_pdf, k=3)

    for i, doc in enumerate(results):
        print(colored(f"【Rank {i+1}】 (Page {doc.metadata.get('page', '?')})", "cyan"))
        print(doc.page_content.replace("\n", " ")[:200] + "...") # 長いので先頭200文字を表示
        print("-" * 60)
else:
    print("PDFデータベースが構築されていません。")